In [11]:
import os
import pandas as pd
import numpy as np
import re

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

In [2]:
folder = "/home/laura/ANALYSIS/VARIANT_CALLING/MTB_ANC_2020/VCF/"



In [3]:
def import_VCF_to_pandas(vcf_file):
    header_lines = 0
    with open(vcf_file) as f:
        first_line = f.readline().strip()
        next_line = f.readline().strip()
        while next_line.startswith("##"):
            header_lines = header_lines + 1
            #print(next_line)
            next_line = f.readline()

    if first_line.startswith('##'):
        df = pd.read_csv(vcf_file, sep='\t', skiprows=[header_lines], header=header_lines)
        
        df['ALT']=df['ALT'].str.upper()
        df['REF']=df['REF'].str.upper()
        #Check INFO
        if 'INFO' in df.columns:
            return df
        else:
            last_column = df.columns[-1]
            df = df.rename(columns={last_column: 'INFO'})
            return df
    else:
        print("This vcf file is not properly formatted")
        sys.exit(1)

In [53]:
#import_VCF42_to_pandas("/home/laura/ANALYSIS/VARIANT_CALLING/MTB_ANC_2020/VCF/10082989-0-COL3.combined.hf.SNP.final.vcf").head()

In [107]:
def handle_polymorphism_freebayes(df):
    for index, _ in df[df.len_AD > 2].iterrows():
        split_AD_all = df.loc[index, 'AD'].split(",")
        split_AD = split_AD_all[1:]
        split_AD = [int(x) for x in split_AD]
        maxAD = max(split_AD)
        max_index = split_AD.index(maxAD)#Obtain index from highest value in list of positions
        df.loc[index, 'len_AD'] = 2 #reset number of alternatives as normal
        df.loc[index, 'AD'] = split_AD_all[0] + ',' + str(maxAD)
        repare_headers = ['ALT', 'AF', 'LEN', 'TYPE', 'ALT_QUAL', 'ALT_DP']
        for header in repare_headers:        
            df.loc[index, header] = df.loc[index, header].split(",")[max_index]  #split bases into list and retrieve the base using ps index

def import_VCF42_freebayes_to_df(vcf_file, sep='\t'):
    """
    Script to read vcf 4.2
    - now handle correct allele frequency calculated by summing REF reads + ALT reads instead from DP parameter
    - now retrieve the largest read number for ALT allele frequency in case is a heterozygous SNP (depends on calculate_ALT_AD())
    - now uses dataframe.iterrows() instead dataframe.index
    - remove snps with two alternate alleles, keeping the most abundant if this is more at least 3 times more frequent
    """

    header_lines = 0
    if vcf_file.endswith(".gz"):
        with gzip.open(vcf_file, 'rb') as f:
            first_line = f.readline().decode().strip()
            next_line = f.readline().decode().strip()
            while next_line.startswith("##"):
                header_lines = header_lines + 1
                next_line = f.readline().decode().strip()
    else:
        with open(vcf_file, 'r') as f:
            first_line = f.readline().strip()
            next_line = f.readline().strip()
            while next_line.startswith("##"):
                header_lines = header_lines + 1
                next_line = f.readline().strip()
    
    if first_line.endswith('VCFv4.2'):
        
        #Use first line as header
        if vcf_file.endswith(".gz"):
            df = pd.read_csv(vcf_file, compression='gzip', sep=sep, skiprows=[header_lines], header=header_lines)
        else:
            df = pd.read_csv(vcf_file, sep=sep, skiprows=[header_lines], header=header_lines)

        sample = df.columns[-1]
        df.rename(columns={sample:'sample'}, inplace=True)
        
        for index, data_row in df.iterrows():
            info_fields = [x.split('=')[0] for x in data_row.INFO.split(';')]
            info_values = [x.split('=')[1] for x in data_row.INFO.split(';')]
            
            format_fields = data_row['FORMAT'].split(":")
            format_values = data_row['sample'].split(":")
                                    
            for ifield, ivalue in zip(info_fields,info_values):
                df.loc[index,ifield] = ivalue
                
            for ffield, fvalue in zip(format_fields,format_values):
                df.loc[index,ffield] = fvalue

        #Rename columns to match iVar output to adapt covidma flow
        df.rename(columns={'#CHROM':'REGION','RO':'REF_DP', 'DP':'TOTAL_DP', 'AO':'ALT_DP', 'QR':'REF_QUAL', 'QA':'ALT_QUAL'}, inplace=True)
        
        df['len_AD'] = df['AD'].str.split(",").str.len()
        
        # this step remove false snps from cohort calling and reset index
        #df = df[df.ALT_AD > 0].reset_index(drop=True)

        handle_polymorphism_freebayes(df) #Leave the most common variation when teo alternatives are present        

        to_float = ['QUAL', 'AN', 'TOTAL_DP','GQ', 'REF_QUAL', 'ALT_QUAL']
        
        to_int = ['POS', 'REF_DP', 'ALT_DP', 'len_AD' ]
        
        to_str = ['REGION','REF','ALT', 'FILTER']
        
        for column in df.columns:
            if column in to_float:
                df[column] = df[column].astype(float)
                
        for column in df.columns:
            if column in to_int:
                df[column] = df[column].astype(int)
                
        for column in df.columns:
            if column in to_str:
                df[column] = df[column].astype(str)
                
        df['REF_FREQ'] = df['REF_DP']/df['TOTAL_DP']
        df['ALT_FREQ'] = df['ALT_DP']/df['TOTAL_DP']
        
        df = df[df.TYPE != 'complex']

        df = df.sort_values(by=['POS']).reset_index(drop=True)
        
    else:
        print("This vcf file is not v4.2")
        sys.exit(1)
           
    return df[['REGION', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'AF', 'AN', 'TOTAL_DP', 'DPB', 'LEN', 
       'NUMALT', 'ODDS', 'TYPE', 'GT', 'AD', 'len_AD', 'REF_DP', 'ALT_DP', 'REF_QUAL', 'ALT_QUAL', 'REF_FREQ', 'ALT_FREQ']]

In [110]:
dffree = import_VCF42_to_pandas("/home/laura/ANALYSIS/provaryota/Variants/var.vcf")

In [106]:
dffree.to_csv('/home/laura/ANALYSIS/provaryota/Variants/var2.tsv', sep="\t", index=False)

In [111]:
dffree.head(20)

,REGION,POS,ID,REF,ALT,QUAL,FILTER,AF,AN,TOTAL_DP,DPB,LEN,NUMALT,ODDS,TYPE,GT,AD,len_AD,REF_DP,ALT_DP,REF_QUAL,ALT_QUAL,REF_FREQ,ALT_FREQ
0,MTB_anc,1701,.,T,C,3.441600e+03,.,1,2.0,115.0,115,1,1,162.643,snp,1/1,"0,114",2,0,114,0.0,3896.0,0.000000,0.991304
1,MTB_anc,1962,.,T,G,0.000000e+00,.,0,2.0,54.0,54,1,1,64.5762,snp,0/0,"51,3",2,51,3,1508.0,41.0,0.944444,0.055556
2,MTB_anc,1969,.,T,G,0.000000e+00,.,0,2.0,55.0,55,1,1,57.6,snp,0/0,"50,5",2,50,5,1534.0,66.0,0.909091,0.090909
3,MTB_anc,1970,.,CA,CC,4.294740e-15,.,0,2.0,56.0,56,1,1,63.2546,snp,0/0,"52,4",2,52,4,1628.0,55.0,0.928571,0.071429
4,MTB_anc,1976,.,C,G,0.000000e+00,.,0,2.0,54.0,54,1,1,59.9927,snp,0/0,"50,4",2,50,4,1527.0,58.0,0.925926,0.074074
5,MTB_anc,1978,.,T,G,5.304040e-15,.,0,2.0,54.0,54,1,1,60.0669,snp,0/0,"49,5",2,49,5,1550.0,70.0,0.907407,0.092593
6,MTB_anc,2009,.,C,G,0.000000e+00,.,0,2.0,57.0,57,1,1,68.8316,snp,0/0,"53,4",2,53,4,1720.0,48.0,0.929825,0.070175
7,MTB_anc,2022,.,A,G,2.961020e-15,.,0,2.0,53.0,53,1,1,60.8961,snp,0/0,"48,5",2,48,5,1595.0,60.0,0.905660,0.094340
8,MTB_anc,2040,.,T,G,0.000000e+00,.,0,2.0,48.0,48,1,1,56.4241,snp,0/0,"44,4",2,44,4,1567.0,51.0,0.916667,0.083333
9,MTB_anc,2532,.,C,T,1.211590e+03,.,1,2.0,40.0,40,1,1,60.0569,snp,1/1,"0,40",2,0,40,0.0,1416.0,0.000000,1.000000


In [65]:
dffree.columns

Index(['#CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT',
       'sample', 'AB', 'ABP', 'AC', 'AF', 'AN', 'AO', 'CIGAR', 'DP', 'DPB',
       'DPRA', 'EPP', 'EPPR', 'GTI', 'LEN', 'MEANALT', 'MQM', 'MQMR', 'NS',
       'NUMALT', 'ODDS', 'PAIRED', 'PAIREDR', 'PAO', 'PQA', 'PQR', 'PRO', 'QA',
       'QR', 'RO', 'RPL', 'RPP', 'RPPR', 'RPR', 'RUN', 'SAF', 'SAP', 'SAR',
       'SRF', 'SRP', 'SRR', 'TYPE', 'technology.ILLUMINA', 'GT', 'AD', 'GL',
       'len_AD', 'REF_AD', 'ALT_AD', 'gt0', 'gt1', 'dp', 'aF'],
      dtype='object')

In [52]:
'NS' in dffree.columns

True

In [113]:
dffree[dffree.ALT_DP < 2]

,REGION,POS,ID,REF,ALT,QUAL,FILTER,AF,AN,TOTAL_DP,DPB,LEN,NUMALT,ODDS,TYPE,GT,AD,len_AD,REF_DP,ALT_DP,REF_QUAL,ALT_QUAL,REF_FREQ,ALT_FREQ


In [99]:
dffree.AF.value_counts()

0      573
0.5    214
1      151
Name: AF, dtype: int64

In [90]:
dffree[dffree.TYPE == 'complex']

,REGION,POS,ID,REF,ALT,QUAL,FILTER,AC,AF,AN,DP,DPB,LEN,NUMALT,ODDS,TYPE,GT,AD,len_AD,REF_DP,ALT_DP,REF_QUAL,ALT_QUAL,REF_FREQ,ALT_FREQ
43,MTB_anc,20285,.,TGGGC,GGGGG,6.222060e-04,.,1,0.5,2.0,11.0,11,5,1,8.85072,complex,0/1,"9,2",2,9,2,335.0,51.0,0.818182,0.181818
61,MTB_anc,32167,.,TGGC,GGGG,1.338800e-14,.,0,0,2.0,34.0,34,4,1,40.629,complex,0/0,"31,2",2,31,2,940.0,37.0,0.911765,0.058824
111,MTB_anc,58973,.,TCCCG,GCCCC,4.106150e-08,.,0,0,2.0,20.0,20.8,5,1,18.4767,complex,0/0,"15,2",2,15,2,400.0,43.0,0.750000,0.100000
160,MTB_anc,81389,.,GA,TT,3.840570e-07,.,0,0,2.0,15.0,15,2,1,16.241,complex,0/0,"11,2",2,11,2,407.0,30.0,0.733333,0.133333
260,MTB_anc,145183,.,ACCA,CCCC,3.533330e-14,.,"0,0","0,0",2.0,35.0,35,4,2,34.1707,complex,0/0,"28,3",2,28,3,987.0,54.0,0.800000,0.085714
267,MTB_anc,145450,.,ATTG,GTTT,2.856370e-13,.,"0,0","0,0",2.0,31.0,31,4,2,31.0102,complex,0/0,"24,2",2,24,2,721.0,38.0,0.774194,0.064516
300,MTB_anc,174677,.,ACCAA,CCCAC,3.581410e-06,.,"0,0","0,0",2.0,18.0,18,5,2,14.0446,complex,0/0,"12,2",2,12,2,404.0,46.0,0.666667,0.111111
308,MTB_anc,175919,.,TCGGA,GCGGG,1.370800e-04,.,"1,0","0.5,0",2.0,15.0,15,5,2,10.3743,complex,0/1,"10,3",2,10,3,349.0,51.0,0.666667,0.200000
309,MTB_anc,175930,.,TTTTC,GTTTT,1.121530e-07,.,0,0,2.0,22.0,22.8,5,1,17.472,complex,0/0,"18,3",2,18,3,656.0,63.0,0.818182,0.136364
319,MTB_anc,177896,.,GGGGC,TGGGG,1.266130e-04,.,1,0.5,2.0,12.0,12,5,1,10.4429,complex,0/1,"9,2",2,9,2,329.0,46.0,0.750000,0.166667


In [1]:
lwfile = '/home/laura/ANALYSIS/provaryota/Variants/freebayes_raw/ALM93896B2COL31.vcf'

In [29]:
def import_VCF40_lofreq_to_df(vcf_file, sep='\t'):
    """
    Script to read vcf 4.0
    - now handle correct allele frequency calculated by summing REF reads + ALT reads instead from DP parameter
    - now retrieve the largest read number for ALT allele frequency in case is a heterozygous SNP (depends on calculate_ALT_AD())
    - now uses dataframe.iterrows() instead dataframe.index
    - remove snps with two alternate alleles, keeping the most abundant if this is more at least 3 times more frequent
    """

    header_lines = 0
    if vcf_file.endswith(".gz"):
        with gzip.open(vcf_file, 'rb') as f:
            first_line = f.readline().decode().strip()
            next_line = f.readline().decode().strip()
            while next_line.startswith("##"):
                header_lines = header_lines + 1
                next_line = f.readline().decode().strip()
    else:
        with open(vcf_file, 'r') as f:
            first_line = f.readline().strip()
            next_line = f.readline().strip()
            while next_line.startswith("##"):
                header_lines = header_lines + 1
                next_line = f.readline().strip()
    
    if first_line.endswith('VCFv4.0'):
        
        #Use first line as header
        if vcf_file.endswith(".gz"):
            df = pd.read_csv(vcf_file, compression='gzip', sep=sep, skiprows=[header_lines], header=header_lines)
        else:
            df = pd.read_csv(vcf_file, sep=sep, skiprows=[header_lines], header=header_lines)
    else:
        print("This vcf file is not v4.0")
        sys.exit(1)

    for index, data_row in df.iterrows():
            info_fields = [x.split('=')[0] for x in data_row.INFO.split(';')]
            info_values = [x.split('=')[1] for x in data_row.INFO.split(';')]

                                    
            for ifield, ivalue in zip(info_fields,info_values):
                df.loc[index,ifield] = ivalue

    df[['REFF', 'REFR', 'ALTF', 'ALTR']] = df['DP4'].str.split(',', expand=True)

    to_int = ['POS', 'REFF', 'REFR', 'ALTF', 'ALTR' ]

    for column in df.columns:
        if column in to_int:
            df[column] = df[column].astype(int)

    df = df.replace('PASS', True)
                
    df.rename(columns={'#CHROM':'REGION', 'DP':'TOTAL_DP', 'AF': 'ALT_FREQ', 'FILTER' : 'PASS'}, inplace=True)

    df['REF_DP'] = df.REFF + df.REFR
    df['ALT_DP'] = df.ALTF + df.ALTR

    return df
    
    

In [30]:
df = import_VCF42_lofreq_to_df(lwfile)

In [31]:
df.head()

,REGION,POS,ID,REF,ALT,QUAL,PASS,INFO,TOTAL_DP,ALT_FREQ,SB,DP4,REFF,REFR,ALTF,ALTR,REF_DP,ALT_DP
0,MTB_anc,1701,.,T,C,3955,True,"DP=118;AF=1.000000;SB=0;DP4=0,0,54,64",118,1.000000,0,"0,0,54,64",0,0,54,64,0,118
1,MTB_anc,2532,.,C,T,1866,True,"DP=55;AF=0.981818;SB=0;DP4=1,0,36,18",55,0.981818,0,"1,0,36,18",1,0,36,18,1,54
2,MTB_anc,8040,.,G,A,2921,True,"DP=80;AF=1.000000;SB=0;DP4=0,0,30,50",80,1.000000,0,"0,0,30,50",0,0,30,50,0,80
3,MTB_anc,9143,.,C,T,956,True,"DP=27;AF=1.000000;SB=0;DP4=0,0,19,8",27,1.000000,0,"0,0,19,8",0,0,19,8,0,27
4,MTB_anc,13460,.,G,A,1069,True,"DP=31;AF=1.000000;SB=0;DP4=0,0,11,20",31,1.000000,0,"0,0,11,20",0,0,11,20,0,31


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1195 entries, 0 to 1194
Data columns (total 16 columns):
REGION    1195 non-null object
POS       1195 non-null int64
ID        1195 non-null object
REF       1195 non-null object
ALT       1195 non-null object
QUAL      1195 non-null int64
FILTER    1195 non-null object
INFO      1195 non-null object
DP        1195 non-null object
AF        1195 non-null object
SB        1195 non-null object
DP4       1195 non-null object
REFF      1195 non-null int64
REFR      1195 non-null int64
ALTF      1195 non-null int64
ALTR      1195 non-null int64
dtypes: int64(6), object(10)
memory usage: 149.5+ KB
